In [83]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as logr
from datetime import datetime
import os
import numpy as np
print(os.getcwd())

/Users/leof/Desktop/Final Data Set - Toronto/Challenge 2 - Future Mine Maintenance


In [84]:
downtimetruck = pd.read_csv("p3/wod/dot2016.csv", encoding = "ISO-8859-1")
cycle = pd.read_csv("p1/2_cycle/TruckCyclesCube v1.csv", encoding = "ISO-8859-1", skipfooter=1 )
downtimetruck['truckno'] = downtimetruck['Equipo'].str.replace("TK","").astype(float)
cycle['truckno'] = cycle["Row Labels"].astype(float)
downtimetruck['maint_date'] = pd.to_datetime(downtimetruck['Start Time'].str[0:8],\
                                             infer_datetime_format=True, errors='coerce')
cycle['date'] = '2016-' + cycle['Month'].map(str) +'-'+ cycle['Day'].map(str)
cycle['date'] = pd.to_datetime(cycle['date'],infer_datetime_format=True, errors='coerce')
del cycle['Row Labels']
del cycle['Month']
del cycle['Day']
#del cycle['date']

/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [85]:
cycle.head()

,Payloads,Paytons,CycleTime_Sum_Minutes,Traveling_Sum_Minutes,Waiting_Sum_Minutes,Spotting_Sum_Minutes,Loading_Sum_Minutes,Hauling_Sum_Minutes,Queued_Sum_Minutes,Backing_Sum_Minutes,Tipping_Sum_Minutes,truckno,date
0,1,162.199997,33.216667,14.883333,1.316667,0.350000,3.466667,12.283333,0.033333,0.366667,0.516667,403.0,2016-02-01
1,2,345.799988,32.316667,15.516667,0.866667,0.200000,2.633333,12.483333,0.283333,0.333333,0.000000,403.0,2016-02-01
2,1,163.399994,26.400000,5.866667,6.166667,0.200000,2.416667,10.883333,0.000000,0.400000,0.466667,403.0,2016-02-01
3,4,729.299988,180.483333,88.550000,5.266667,2.283333,11.883333,65.916667,2.366667,2.133333,2.083333,403.0,2016-02-01
4,3,546.400009,107.933333,39.300000,7.283333,0.433333,6.566667,50.050000,0.816667,1.266667,2.216667,403.0,2016-02-01


In [86]:
downtimetruck = downtimetruck[downtimetruck['Work Type'] == 'No programado'][['maint_date','truckno','Duration (min)','System_google_translated']]
downtimetruck.head()
downtimetruck_maint_only = downtimetruck[['maint_date','truckno']].drop_duplicates()


In [87]:
downtimetruck_maint_only['maint'] = 1
downtimetruck_maint_only.head()

,maint_date,truckno,maint
0,2016-01-01,408.0,1
1,2016-01-02,408.0,1
2,2016-01-02,403.0,1
5,2016-01-03,408.0,1
7,2016-01-03,434.0,1


In [88]:
#df2 is whether or not maint happened
df2 = cycle.merge(downtimetruck_maint_only, left_on=['truckno','date'], right_on=['truckno','maint_date'], how='left')
df2['maint_date_prev'] = df2.groupby(['truckno'])['maint_date'].shift(1)
df2['days_since_last_maint'] = (df2.maint_date -df2.maint_date_prev ).dt.days
df2.loc[df2['days_since_last_maint'].isnull(), 'days_since_last_maint'] = 0
df2['days_since_start'] = (df2.date - datetime(2016, 1, 1)).dt.days
df2.loc[df2['maint'].isnull(), 'maint'] = 0


del df2['maint_date']
del df2['maint_date_prev']
del df2['date']

df2['is_train'] = np.random.uniform(0, 1, len(df2)) <= .55

train2, test2 = df2[df2['is_train']==True], df2[df2['is_train']==False]

features_train2 = train2.ix[:, train2.columns != 'maint']
features_test2 = test2.ix[:, test2.columns != 'maint']

y_train2 = train2['maint']
y_test2 = test2['maint']

#df2.head()

from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_jobs=4, random_state=0, n_estimators=1000)
classifier2 = clf2.fit(features_train2, y_train2)
predictions2 = classifier2.predict_proba(features_test2)


In [90]:
#test model perf
((predictions2[:,1]>0.5).tolist() == (y_test2==1)).mean()

0.85114401076716018

In [6]:
df = cycle.merge(downtimetruck, left_on=['truckno','date'], right_on=['truckno','maint_date'], how='left')
df.loc[df['Duration (min)'].isnull(), 'Duration (min)'] = 0
df['maint_date_cat_prev'] = df.groupby(['truckno','System_google_translated'])['maint_date'].shift(1)
df['days_since_last_maint_cat'] = (df.maint_date -df.maint_date_cat_prev ).dt.days
df.loc[df['days_since_last_maint_cat'].isnull(), 'days_since_last_maint_cat'] = 0

df['maint_date_prev'] = df.groupby(['truckno'])['maint_date'].shift(1)
df['days_since_last_maint'] = (df.maint_date -df.maint_date_prev ).dt.days
df.loc[df['days_since_last_maint'].isnull(), 'days_since_last_maint'] = 0

df['days_since_start'] = (df.date - datetime(2016, 1, 1)).dt.days

In [7]:
#df = df.join( pd.get_dummies(df.System_google_translated))
#del df['System_google_translated']
del df['maint_date']
del df['maint_date_cat_prev']
del df['maint_date_prev']

df.dtypes

Payloads                              int64
Paytons                             float64
CycleTime_Sum_Minutes               float64
Traveling_Sum_Minutes               float64
Waiting_Sum_Minutes                 float64
Spotting_Sum_Minutes                float64
Loading_Sum_Minutes                 float64
Hauling_Sum_Minutes                 float64
Queued_Sum_Minutes                  float64
Backing_Sum_Minutes                 float64
Tipping_Sum_Minutes                 float64
truckno                             float64
date                         datetime64[ns]
Duration (min)                      float64
System_google_translated             object
days_since_last_maint_cat           float64
days_since_last_maint               float64
days_since_start                      int64
dtype: object

In [8]:
df.loc[df['System_google_translated'].isnull(), 'System_google_translated'] = "No Maintenance"
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .55
del df['date']
del df['Duration (min)']

train, test = df[df['is_train']==True], df[df['is_train']==False]


In [9]:
features_train = train.ix[:, train.columns != 'System_google_translated']
features_test = test.ix[:, test.columns != 'System_google_translated']

y_train, y_train_lab = pd.factorize(train['System_google_translated'])
y_test, y_test_lab = pd.factorize(test['System_google_translated'])



In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=4, random_state=0, n_estimators=1000)
classifier = clf.fit(features_train, y_train)
predictions = classifier.predict_proba(features_test)

In [11]:
preds_labels = pd.DataFrame(predictions, columns=y_train_lab)
preds_labels['row_max'] = preds_labels.max(axis=1)


for column in preds_labels.ix[:, preds_labels.columns != 'row_max']:
    preds_labels.loc[preds_labels[column] == preds_labels['row_max'], column] = 9
    #preds_labels.loc[preds_labels[column] != preds_labels['row_max'], 'z'] = 0


for column in preds_labels.ix[:, preds_labels.columns != 'row_max']:
    #preds_labels.loc[preds_labels[column] != 9, column] = 0
    #preds_labels.loc[preds_labels[column] == 9, column] = 1
    preds_labels.loc[preds_labels[column] == 9, 'predicted_maint_cat'] = column


del preds_labels['row_max']

In [12]:
test['predicted_maint_cat'] = preds_labels['predicted_maint_cat'].values
test['correct_pred'] = test['System_google_translated'] == test['predicted_maint_cat']

/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
#how many category do you get right?
test[test.System_google_translated != 'No Maintenance'].correct_pred.mean()

0.11290322580645161

In [14]:
#how many predictions are right
test.correct_pred.mean()

0.79513888888888884

In [15]:
#how many times do i predict that maint is necessary correctly?
test['predict_main_necessary'] = test.predicted_maint_cat == 'No Maintenance'
test['main_necessary'] = test.System_google_translated == 'No Maintenance'
test['predict_that_maint_nec_at_all'] = test['predict_main_necessary'] == test['main_necessary']

/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/leof/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [16]:
#Predict that maintenance is indeed necessary
test.predict_that_maint_nec_at_all.mean()

0.88020833333333337

In [17]:
test_days_only = test[['truckno','days_since_start','main_necessary','predict_that_maint_nec_at_all']].drop_duplicates()

In [18]:
#number of maintenance days actual by truck
test_days_only.groupby('truckno')['main_necessary'].sum()

truckno
403.0    261.0
408.0    258.0
422.0    266.0
434.0    243.0
Name: main_necessary, dtype: float64

In [19]:
#number of maintenance days pred by truck
test_days_only.groupby('truckno')['predict_that_maint_nec_at_all'].sum()

truckno
403.0    295.0
408.0    291.0
422.0    288.0
434.0    287.0
Name: predict_that_maint_nec_at_all, dtype: float64

In [20]:
#overall how oftendoes what need unplanned maintenance?
test[['truckno','days_since_start','System_google_translated']].drop_duplicates().groupby(['truckno','System_google_translated']).count()

days_since_start
truckno System_google_translated                  
403.0   AIR                                     10
        BRAKES                                   4
        CAB / CHASSIS                            9
        ELECTRIC 12/24 VOLTS                     5
        HYDRAULIC                               10
        IMPLEMENTS                               1
        LUBRICATION                              1
        MOTOR                                    8
        No Maintenance                         261
        POWER TRAIN                              4
        RUBBER, RIMS AND TIRES                   8
        SECURITY                                 7
        SERVICES                                 1
408.0   ADDRESS                                  1
        AIR                                     15
        BRAKES                                   2
        CAB / CHASSIS                           17
        ELECTRIC 12/24 VOLTS                     7
        HYDRAULIC                                5
        LOW VOLTAGE                              1
        LUBRICATION                              1
        MOTOR                                    6
        No Maintenance                         258
        POWER TRAIN                              1
        PROPULSION                               1
        RUBBER, RIMS AND TIRES                   7
        SECURITY                                 5
        SERVICES                                 1
422.0   ADDRESS                                  1
        AIR                                      1
        BRAKES                                   2
        CAB / CHASSIS                            9
        CABIN / CHASSIS                          1
        ELECTRIC 12/24 VOLTS                     8
        HYDRAULIC                                2
        IMPLEMENTS                               1
        LOW VOLTAGE                              1
        MOTOR                                    8
        No Maintenance                         266
        POWER TRAIN                              3
        RUBBER, RIMS AND TIRES                  12
        SECURITY                                 2
        SERVICES                                 3
434.0   ADDRESS                                  1
        AIR                                      1
        BRAKES                                  23
        CAB / CHASSIS                           16
        ELECTRIC 12/24 VOLTS                     9
        HYDRAULIC                                8
        IMPLEMENTS                               3
        LUBRICATION                              2
        MOTOR                                    8
        No Maintenance                         242
        POWER TRAIN                              2
        RUBBER, RIMS AND TIRES                  18
        SECURITY                                 3

In [21]:
test[['truckno','days_since_start','System_google_translated','correct_pred']].drop_duplicates().groupby(['System_google_translated'])['correct_pred'].mean()

System_google_translated
ADDRESS                   0.000000
AIR                       0.071429
BRAKES                    0.138889
CAB / CHASSIS             0.272727
CABIN / CHASSIS           0.000000
ELECTRIC 12/24 VOLTS      0.096774
HYDRAULIC                 0.076923
IMPLEMENTS                0.000000
LOW VOLTAGE               0.000000
LUBRICATION               0.000000
MOTOR                     0.216216
No Maintenance            0.999027
POWER TRAIN               0.090909
PROPULSION                0.000000
RUBBER, RIMS AND TIRES    0.117647
SECURITY                  0.150000
SERVICES                  0.000000
Name: correct_pred, dtype: float64